In [1]:
import pandas as pd

In [2]:
item_df = pd.read_csv('../steam_games.csv', sep=';')

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3098: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Curation

- Type: game
- Genre: Indie (중복 고려)
- Languages: Korean
- Reviews: 긍정적인 리뷰가 전체 리뷰의 90% 이상 && 리뷰 수 100개 이상
- Tags: 선정성 게임 제외

- POP: PR_rate
- 장르별: Genre
- 카테고리별(멀티플레이 등): Categories
- 가격대: Initial Price
- 플랫폼: Platforms

In [3]:
item_df.columns

Index(['App ID', 'Name', 'Short Description', 'Developer', 'Publisher',
       'Genre', 'Tags', 'Type', 'Categories', 'Owners', 'Positive Reviews',
       'Negative Reviews', 'Price', 'Initial Price', 'Discount', 'CCU',
       'Languages', 'Platforms', 'Release Date', 'Required Age', 'Website',
       'Header Image'],
      dtype='object')

In [4]:
item = item_df.copy()
item = item[['App ID', 'Name', 'Genre', 'Type', 'Languages', 'Categories', 'Initial Price', 'Platforms', 'Tags', 'Positive Reviews', 'Negative Reviews']]
item['Reviews'] = item['Positive Reviews'] + item['Negative Reviews']
item['PR_rate'] = item['Positive Reviews'] / item['Reviews']
print(len(item))

55691


## Filtering

### Type

In [5]:
item.Type.unique()

array(['game', 'hardware'], dtype=object)

In [6]:
item = item[item.Type == 'game']
print(len(item))

55690


### Genre

In [7]:
item.Genre.unique()  # Indie

array(['Action', 'Action, Adventure, Indie',
       'Adventure, Indie, RPG, Strategy', ...,
       'Action, Free to Play, Indie, Massively Multiplayer, Racing, RPG, Simulation',
       'Action, Indie, Massively Multiplayer, RPG, Strategy, Early Access',
       'Adventure, Casual, Free to Play, Massively Multiplayer, Strategy'],
      dtype=object)

In [8]:
item = item[item.Genre.str.contains('Indie', na=False)]
print(len(item))

39681


### Languages

In [9]:
item.Languages.unique()

array(['English, Korean, Simplified Chinese',
       'Simplified Chinese, English, Japanese, Traditional Chinese, French, German, Spanish - Spain, Russian, Portuguese - Brazil',
       'English', ...,
       'English, Spanish - Spain, Japanese, Korean, Polish, Portuguese - Brazil, Russian, French, German, Simplified Chinese, Traditional Chinese',
       'English, French, Italian, German, Spanish - Spain, Japanese, Portuguese - Portugal, Simplified Chinese, Traditional Chinese',
       'English, German, Spanish - Spain, Japanese, Polish, Russian'],
      dtype=object)

In [10]:
item = item[item.Languages.str.contains("Korean", na=False)]
print(len(item))

4542


### Reviews

In [11]:
item = item[item.PR_rate >= 0.9]
print(len(item))

1555


In [12]:
item = item[item.Reviews >= 100]
print(len(item))

779


### Tags

In [13]:
item.Tags.unique()  # Nudity, Sexual Content, ...

array(['Casual: 219, Puzzle: 214, Colorful: 186, Abstract: 176, Psychedelic: 173, VR: 173, Stylized: 151, Controller: 141, PvP: 125, Score Attack: 114, Tutorial: 102, Co-op: 100, Local Co-Op: 82, Local Multiplayer: 69, Multiplayer: 59, Great Soundtrack: 48, Singleplayer: 42, Indie: 38, Online Co-Op: 36, Beautiful: 26',
       'Nudity: 47, Sexual Content: 45, Casual: 25, Anime: 22, Indie: 22, Simulation: 16, RPG: 13, Mature: 12, Puzzle: 12',
       'Casual: 207, Sports: 200, Life Sim: 174, Surreal: 174, Funny: 173, Psychedelic: 164, Dog: 162, 2D: 157, Indie: 154, Minimalist: 154, Hand-drawn: 152, Family Friendly: 143, Philosophical: 136, Psychological: 134, Singleplayer: 130, Colorful: 124, Immersive Sim: 122, Physics: 117, e-sports: 111, Action: 81',
       'Indie: 96, Action: 95, Bullet Hell: 25, Cute: 21, Fighting: 18, Anime: 17, Female Protagonist: 17, Multiplayer: 17, "Shoot Em Up": 17, 2D: 15, Local Multiplayer: 14, 2D Fighter: 13, Difficult: 13, PvP: 13, 2.5D: 12, Arcade: 12, Col

In [14]:
item = item[~item.Tags.str.contains("Nudity|Sexual", na=False)]
print(len(item))

736


In [15]:
item = item.reset_index(drop=True)
item.to_csv('filtered.csv', index=False)

## URL Check

In [16]:
import requests

In [17]:
url = 'https://store.steampowered.com/app/'
error_items = []

for i, n in item[['App ID', 'Name']].values:
    data = requests.get(url+str(i))
    if n not in data.content.decode():
        error_items.append(i)

In [18]:
item[item['App ID'].isin(error_items)]  # 필터링된 아이템 셋에는 문제가 없는 것으로 확인

,App ID,Name,Genre,Type,Languages,Categories,Initial Price,Platforms,Tags,Positive Reviews,Negative Reviews,Reviews,PR_rate
83,1161160,talesshop puzzle 테일즈샵 퍼즐,"Casual, Free to Play, Indie",game,"English, Not supported, Korean","Single-player, Steam Achievements, Steam Cloud",0,windows,"Casual: 37, Free to Play: 35, Indie: 35, Puzzl...",279,18,297,0.939394
210,1452540,Frincess&Cnight,"Action, Adventure, Indie",game,"English, Korean, Japanese, Simplified Chinese","Multi-player, Single-player, Co-op, Steam Achi...",399,"windows, mac","Action: 64, Puzzle: 50, Platformer: 41, Pixel ...",104,4,108,0.962963
724,972660,Spiritfarer: Farewell Edition,"Adventure, Indie, Simulation",game,"English, French, Italian, German, Spanish - Sp...","Multi-player, Single-player, Co-op, Steam Achi...",2999,"windows, mac, linux","Emotional: 391, Story Rich: 374, Indie: 368, M...",22741,1000,23741,0.957879


## Producing

### POP(PR_rate)

In [19]:
item.sort_values(by=['Reviews', 'PR_rate'], ascending=False).head(10).style.hide_index()
#['App ID'].values

App ID,Name,Genre,Type,Languages,Categories,Initial Price,Platforms,Tags,Positive Reviews,Negative Reviews,Reviews,PR_rate
4000,Garry's Mod,"Indie, Simulation",game,"English, French, Italian, German, Spanish - Spain, Bulgarian, Czech, Danish, Dutch, Finnish, Greek, Hungarian, Japanese, Korean, Norwegian, Polish, Portuguese - Portugal, Portuguese - Brazil, Russian, Simplified Chinese, Swedish, Thai, Traditional Chinese, Turkish, Ukrainian","Multi-player, Single-player, Valve Anti-Cheat enabled, Co-op, Captions available, Includes level editor, Steam Achievements, Steam Cloud, Cross-Platform Multiplayer, Steam Trading Cards, Steam Workshop, Online PvP, Online Co-op, Remote Play on Tablet, LAN PvP, LAN Co-op, PvP",999,"windows, mac, linux","Sandbox: 17628, Multiplayer: 13726, Funny: 12249, Moddable: 9861, Building: 9610, Comedy: 8298, Mod: 8239, Co-op: 8167, First-Person: 7937, Physics: 6867, Simulation: 6650, Online Co-Op: 6469, FPS: 6127, Singleplayer: 6055, Action: 5509, Animation & Modeling: 5339, Shooter: 5339, Indie: 4975, Adventure: 4651, Massively Multiplayer: 4650",861240,29998,891238,0.966341
227300,Euro Truck Simulator 2,"Indie, Simulation",game,"English, German, French, Italian, Korean, Spanish - Spain, Simplified Chinese, Russian, Japanese, Portuguese - Portugal, Polish, Danish, Dutch, Finnish, Norwegian, Swedish, Hungarian, Czech, Romanian, Turkish, Portuguese - Brazil, Bulgarian, Greek, Ukrainian","Multi-player, Single-player, Co-op, Includes level editor, Partial Controller Support, Steam Achievements, Steam Cloud, Steam Trading Cards, Steam Workshop, Online Co-op, Remote Play on Phone, Remote Play on Tablet, Remote Play Together",1999,"windows, mac, linux","Driving: 12059, Transportation: 12044, Simulation: 11989, Open World: 11951, Automobile Sim: 11939, Realistic: 11896, Moddable: 11836, Relaxing: 11834, Immersive Sim: 11783, Exploration: 11765, Economy: 11760, Management: 11743, Atmospheric: 11724, Multiplayer: 3978, Controller: 3740, Family Friendly: 3722, Third Person: 3721, Casual: 3655, Adventure: 3651, Indie: 3642",572368,15615,587983,0.973443
431960,Wallpaper Engine,"Casual, Indie, Animation & Modeling, Design & Illustration, Photo Editing, Utilities",game,"English, German, Simplified Chinese, Polish, Portuguese - Brazil, French, Italian, Spanish - Spain, Finnish, Japanese, Korean, Swedish, Thai, Traditional Chinese, Turkish, Portuguese - Portugal, Russian, Danish, Czech, Arabic, Ukrainian, Indonesian, Persian","Includes level editor, Steam Achievements, Steam Trading Cards, Steam Workshop, Steam Workshop",399,windows,"Mature: 2483, Utilities: 1923, Software: 1541, Anime: 1430, Design & Illustration: 1155, Animation & Modeling: 926, Memes: 570, Indie: 556, Cute: 504, NSFW: 478, Action: 426, Funny: 424, Early Access: 423, Singleplayer: 390, Photo Editing: 290, Sandbox: 273, Game Development: 194, Horror: 165, First-Person: 127, Gaming: 71",561096,11031,572127,0.980719
413150,Stardew Valley,"Indie, RPG, Simulation",game,"English, German, Spanish - Spain, Japanese, Portuguese - Brazil, Russian, Simplified Chinese, French, Italian, Hungarian, Korean, Turkish","Multi-player, Single-player, Co-op, Steam Achievements, Steam Cloud, Shared/Split Screen, Full controller support, Steam Trading Cards, Online Co-op, Shared/Split Screen Co-op, Remote Play on Phone, Remote Play on Tablet, Remote Play Together, LAN Co-op",1499,"windows, mac, linux","Farming Sim: 5804, Life Sim: 5607, Pixel Graphics: 5470, RPG: 5274, Multiplayer: 5058, Relaxing: 4623, Agriculture: 4445, Simulation: 4407, Crafting: 3638, Sandbox: 3398, Indie: 3115, Building: 2894, Singleplayer: 2635, Casual: 2546, Open World: 2467, 2D: 2249, Cute: 2082, Great Soundtrack: 2080, Dating Sim: 1910, Fishing: 1742",497558,9283,506841,0.981685
739630,Phasmophobia,"Action, Indie, Early Access",game,"English, Portuguese - Portugal, Spanish - Spain, Portuguese - Brazil, German, French, Italian, Bulgarian, Czech, Danish, Dutch, Finnish, Greek, Hungarian, Japanese, K

### Genre 별 추천

In [20]:
item.Genre.unique()

array(['Casual, Indie', 'Action, Casual, Indie, Racing, Sports',
       'Action, Indie', 'Adventure, Casual, Indie, Simulation, Strategy',
       'Adventure, Indie', 'Action, Casual, Indie',
       'Action, Casual, Indie, Simulation', 'Indie, Strategy',
       'Casual, Free to Play, Indie', 'Action, Indie, Early Access',
       'Action, Adventure, Indie, RPG', 'Action, Adventure, Indie',
       'Action, Casual, Indie, Sports', 'Adventure, Casual, Indie',
       'Casual, Indie, Strategy', 'Adventure, Casual, Indie, Strategy',
       'Action, Indie, RPG', 'Action, Casual, Indie, Early Access',
       'Indie, Simulation, Strategy, Early Access', 'Indie, Simulation',
       'Casual, Indie, Simulation, Strategy', 'Indie, RPG, Strategy',
       'Indie, Simulation, Strategy', 'Casual, Indie, Simulation, Sports',
       'Casual, Indie, Simulation', 'Adventure, Indie, Strategy',
       'Free to Play, Indie, Early Access', 'Indie',
       'Action, Indie, Simulation', 'Adventure, Indie, RPG',
   

In [21]:
#Action, Adventure, Casual, RPG, Simulation, Racing, Sports, Early Access, Strategy, ...
item[item.Genre.str.contains('Action', na=False)].sort_values(by=['Reviews', 'PR_rate'], ascending=False).head(10).style.hide_index()
#['App ID'].values

App ID,Name,Genre,Type,Languages,Categories,Initial Price,Platforms,Tags,Positive Reviews,Negative Reviews,Reviews,PR_rate
739630,Phasmophobia,"Action, Indie, Early Access",game,"English, Portuguese - Portugal, Spanish - Spain, Portuguese - Brazil, German, French, Italian, Bulgarian, Czech, Danish, Dutch, Finnish, Greek, Hungarian, Japanese, Korean, Norwegian, Polish, Romanian, Russian, Simplified Chinese, Swedish, Traditional Chinese, Turkish, Ukrainian, Catalan","Multi-player, Single-player, Co-op, Steam Cloud, Full controller support, Online Co-op, Remote Play on Phone, Remote Play on Tablet",1399,windows,"Horror: 2719, Online Co-Op: 1764, Multiplayer: 1595, Psychological Horror: 1292, Co-op: 1147, VR: 996, Supernatural: 964, First-Person: 896, Investigation: 802, Dark: 661, Mystery: 587, Demons: 578, Detective: 574, Early Access: 445, 3D: 410, Thriller: 408, Indie: 384, Action: 335, Tactical: 306, Lovecraftian: 142",475555,15543,491098,0.968351
242760,The Forest,"Action, Adventure, Indie, Simulation",game,"English, French, German, Czech, Finnish, Japanese, Korean, Polish, Portuguese - Brazil, Russian, Simplified Chinese, Swedish, Traditional Chinese, Turkish, Italian, Spanish - Spain","Multi-player, Single-player, Co-op, Partial Controller Support, Steam Cloud, Online Co-op, LAN Co-op",1999,windows,"Open World Survival Craft: 11651, Survival: 11283, Open World: 8119, Horror: 7687, Crafting: 6062, Adventure: 5228, Building: 5107, Survival Horror: 4676, First-Person: 4232, Action: 3527, Exploration: 3316, Sandbox: 3125, Atmospheric: 2590, Singleplayer: 2518, Simulation: 2451, Gore: 2202, Realistic: 2184, Indie: 2165, Early Access: 1611, Zombies: 1530",394258,19678,413936,0.952461
367520,Hollow Knight,"Action, Adventure, Indie",game,"English, French, German, Spanish - Spain, Korean, Simplified Chinese, Italian, Portuguese - Brazil, Russian, Japanese","Single-player, Steam Achievements, Steam Cloud, Full controller support, Steam Trading Cards, Remote Play on Phone, Remote Play on Tablet, Remote Play on TV",1499,"windows, mac, linux","Metroidvania: 4203, Souls-like: 3521, Platformer: 3214, Great Soundtrack: 3030, 2D: 2862, Difficult: 2860, Indie: 2498, Atmospheric: 2340, Singleplayer: 2285, Exploration: 2277, Adventure: 2117, Story Rich: 2028, Hand-drawn: 1571, Action: 1501, Dark Fantasy: 1394, Multiple Endings: 1382, Open World: 1262, Cute: 1224, Controller: 948, Side Scroller: 878",246135,7193,253328,0.971606
632360,Risk of Rain 2,"Action, Indie",game,"English, French, Italian, German, Spanish - Spain, Japanese, Korean, Portuguese - Brazil, Russian, Simplified Chinese, Turkish","Multi-player, Single-player, Co-op, Steam Cloud, Steam Leaderboards, Full controller support, Online PvP, Online Co-op, Remote Play on Phone, Remote Play on Tablet, Remote Play on TV, PvP",2499,windows,"Third-Person Shooter: 2610, Action Roguelike: 2400, Multiplayer: 2325, Action: 2067, Rogue-lite: 2006, Co-op: 1979, Looter Shooter: 1824, Rogue-like: 1698, Loot: 1508, Shooter: 1148, Online Co-Op: 1130, Difficult: 1002, Third Person: 982, Indie: 957, Survival: 951, Singleplayer: 940, Soundtrack: 878, Great Soundtrack: 827, Early Access: 703, Bullet Hell: 411",206580,7652,214232,0.964282
1145360,Hades,"Action, Indie, RPG",game,"English, French, Italian, German, Spanish - Spain, Korean, Portuguese - Brazil, Russian, Simplified Chinese, Polish, Japanese","Single-player, Captions available, Steam Cloud, Full controller support, Steam Trading Cards, Remote Play on Phone, Remote Play on Tablet, Remote Play on TV",2499,"windows, mac","Action Roguelike: 978, Rogue-lite: 667, Indie: 659, Hack and Slash: 637, Action: 629, Mythology: 534, RPG: 520, Rogue-like: 486, Great Soundtrack: 483, Singleplayer: 452, Dungeon Crawler: 439, Story Rich: 434, Replay Value: 389, Isometric: 289, Difficult: 277, Action RPG: 219, Hand-drawn: 205, Atmospheric: 192, Perma Death: 81, LGBTQ+: 63",199960,2829,202789,0.986050
1794680,Vampire Survivors,"Action, Casual, Indie, RPG",game,"

### Categories 별 추천

In [22]:
item.Categories.unique()

array(['Multi-player, Single-player, Co-op, Steam Achievements, Steam Cloud, Shared/Split Screen, Steam Leaderboards, Cross-Platform Multiplayer, Full controller support, Online PvP, Shared/Split Screen PvP, Online Co-op, Shared/Split Screen Co-op, Remote Play Together, PvP',
       'Single-player, Stats, Steam Achievements, Steam Cloud',
       'Multi-player, Single-player, Shared/Split Screen, Full controller support, Online PvP, Shared/Split Screen PvP, Remote Play on Phone, Remote Play on Tablet, Remote Play Together, PvP',
       'Single-player, Steam Achievements, Steam Cloud, Full controller support, Steam Trading Cards',
       'Single-player, Steam Achievements, Full controller support',
       'Multi-player, Single-player, Co-op, Steam Achievements, Shared/Split Screen, Steam Leaderboards, Cross-Platform Multiplayer, Full controller support, Steam Trading Cards, Online PvP, Shared/Split Screen PvP, Online Co-op, Shared/Split Screen Co-op, Remote Play on Phone, Remote Play on 

In [23]:
#Single-player, Multi-player, Co-op, PvP, Steam Cloud, ...
item[item.Categories.str.contains('Co-op', na=False)].sort_values(by=['Reviews', 'PR_rate'], ascending=False).head(10).style.hide_index()
#['App ID'].values

App ID,Name,Genre,Type,Languages,Categories,Initial Price,Platforms,Tags,Positive Reviews,Negative Reviews,Reviews,PR_rate
4000,Garry's Mod,"Indie, Simulation",game,"English, French, Italian, German, Spanish - Spain, Bulgarian, Czech, Danish, Dutch, Finnish, Greek, Hungarian, Japanese, Korean, Norwegian, Polish, Portuguese - Portugal, Portuguese - Brazil, Russian, Simplified Chinese, Swedish, Thai, Traditional Chinese, Turkish, Ukrainian","Multi-player, Single-player, Valve Anti-Cheat enabled, Co-op, Captions available, Includes level editor, Steam Achievements, Steam Cloud, Cross-Platform Multiplayer, Steam Trading Cards, Steam Workshop, Online PvP, Online Co-op, Remote Play on Tablet, LAN PvP, LAN Co-op, PvP",999,"windows, mac, linux","Sandbox: 17628, Multiplayer: 13726, Funny: 12249, Moddable: 9861, Building: 9610, Comedy: 8298, Mod: 8239, Co-op: 8167, First-Person: 7937, Physics: 6867, Simulation: 6650, Online Co-Op: 6469, FPS: 6127, Singleplayer: 6055, Action: 5509, Animation & Modeling: 5339, Shooter: 5339, Indie: 4975, Adventure: 4651, Massively Multiplayer: 4650",861240,29998,891238,0.966341
227300,Euro Truck Simulator 2,"Indie, Simulation",game,"English, German, French, Italian, Korean, Spanish - Spain, Simplified Chinese, Russian, Japanese, Portuguese - Portugal, Polish, Danish, Dutch, Finnish, Norwegian, Swedish, Hungarian, Czech, Romanian, Turkish, Portuguese - Brazil, Bulgarian, Greek, Ukrainian","Multi-player, Single-player, Co-op, Includes level editor, Partial Controller Support, Steam Achievements, Steam Cloud, Steam Trading Cards, Steam Workshop, Online Co-op, Remote Play on Phone, Remote Play on Tablet, Remote Play Together",1999,"windows, mac, linux","Driving: 12059, Transportation: 12044, Simulation: 11989, Open World: 11951, Automobile Sim: 11939, Realistic: 11896, Moddable: 11836, Relaxing: 11834, Immersive Sim: 11783, Exploration: 11765, Economy: 11760, Management: 11743, Atmospheric: 11724, Multiplayer: 3978, Controller: 3740, Family Friendly: 3722, Third Person: 3721, Casual: 3655, Adventure: 3651, Indie: 3642",572368,15615,587983,0.973443
413150,Stardew Valley,"Indie, RPG, Simulation",game,"English, German, Spanish - Spain, Japanese, Portuguese - Brazil, Russian, Simplified Chinese, French, Italian, Hungarian, Korean, Turkish","Multi-player, Single-player, Co-op, Steam Achievements, Steam Cloud, Shared/Split Screen, Full controller support, Steam Trading Cards, Online Co-op, Shared/Split Screen Co-op, Remote Play on Phone, Remote Play on Tablet, Remote Play Together, LAN Co-op",1499,"windows, mac, linux","Farming Sim: 5804, Life Sim: 5607, Pixel Graphics: 5470, RPG: 5274, Multiplayer: 5058, Relaxing: 4623, Agriculture: 4445, Simulation: 4407, Crafting: 3638, Sandbox: 3398, Indie: 3115, Building: 2894, Singleplayer: 2635, Casual: 2546, Open World: 2467, 2D: 2249, Cute: 2082, Great Soundtrack: 2080, Dating Sim: 1910, Fishing: 1742",497558,9283,506841,0.981685
739630,Phasmophobia,"Action, Indie, Early Access",game,"English, Portuguese - Portugal, Spanish - Spain, Portuguese - Brazil, German, French, Italian, Bulgarian, Czech, Danish, Dutch, Finnish, Greek, Hungarian, Japanese, Korean, Norwegian, Polish, Romanian, Russian, Simplified Chinese, Swedish, Traditional Chinese, Turkish, Ukrainian, Catalan","Multi-player, Single-player, Co-op, Steam Cloud, Full controller support, Online Co-op, Remote Play on Phone, Remote Play on Tablet",1399,windows,"Horror: 2719, Online Co-Op: 1764, Multiplayer: 1595, Psychological Horror: 1292, Co-op: 1147, VR: 996, Supernatural: 964, First-Person: 896, Investigation: 802, Dark: 661, Mystery: 587, Demons: 578, Detective: 574, Early Access: 445, 3D: 410, Thriller: 408, Indie: 384, Action: 335, Tactical: 306, Lovecraftian: 142",475555,15543,491098,0.968351
242760,The Forest,"Action, Adventure, Indie, Simulation",game,"English, French, German, Czech, Finnish, Japanese, Korean, Polish, Portuguese - Brazil, Russian, Simplified Chinese, Swedish, Traditional Chinese, Turkish, Ital

### Initial Price 별 추천

In [24]:
item['Initial Price'].unique()

array([3999,  299, 1299,  999, 1499, 1999,  799,    0,  699, 1699, 2499,
        499,   99, 1199,  199,  599, 3499,  399, 1799,  899, 2999, 1313,
       1899, 2199, 1599, 1399, 3000, 1099, 1995, 1666])

In [25]:
item[item['Initial Price'] < 100].sort_values(by=['Reviews', 'PR_rate'], ascending=False).head(10).style.hide_index()
#['App ID'].values

App ID,Name,Genre,Type,Languages,Categories,Initial Price,Platforms,Tags,Positive Reviews,Negative Reviews,Reviews,PR_rate
677620,Splitgate: Arena Warfare,"Action, Free to Play, Indie",game,"English, French, Italian, German, Japanese, Polish, Romanian, Russian, Portuguese - Brazil, Spanish - Latin America, Spanish - Spain, Portuguese - Portugal, Turkish, Korean, Traditional Chinese","Multi-player, Single-player, Captions available, Steam Achievements, Cross-Platform Multiplayer, Full controller support, In-App Purchases, Online PvP, PvP",0,"windows, linux","Free to Play: 607, FPS: 457, Multiplayer: 375, Arena Shooter: 370, Shooter: 305, Action: 298, Fast-Paced: 264, First-Person: 223, PvP: 221, Competitive: 182, Sci-fi: 166, Indie: 142, Team-Based: 123, e-sports: 115, Strategy: 102, Tactical: 102, Online Co-Op: 99, Physics: 85, Co-op: 76, Violent: 70",97558,9031,106589,0.915273
714010,Aim Lab,"Action, Adventure, Casual, Free to Play, Indie, Simulation, Early Access",game,"English, Spanish - Spain, Spanish - Latin America, Portuguese - Brazil, French, Italian, German, Japanese, Korean, Polish, Russian, Simplified Chinese, Thai, Traditional Chinese, Turkish","Single-player, Stats, Steam Achievements, Steam Leaderboards, Full controller support, Steam Workshop",0,windows,"Free to Play: 483, FPS: 340, Shooter: 280, Simulation: 279, First-Person: 231, Singleplayer: 231, Action: 220, Multiplayer: 215, Early Access: 210, e-sports: 205, Competitive: 201, Casual: 198, Indie: 195, Sniper: 190, Adventure: 181, Tactical: 177, Fast-Paced: 176, Difficult: 174, Score Attack: 171, Third-Person Shooter: 116",74114,4316,78430,0.944970
265630,Fistful of Frags,"Action, Free to Play, Indie",game,"English, Spanish - Spain, French, Russian, German, Czech, Hungarian, Polish, Portuguese - Portugal, Swedish, Simplified Chinese, Portuguese - Brazil, Traditional Chinese, Korean, Italian","Multi-player, Co-op, Stats, Includes Source SDK, Includes level editor, Steam Achievements, Steam Leaderboards, Cross-Platform Multiplayer, Steam Workshop, Online PvP, Online Co-op, Remote Play Together, LAN PvP, LAN Co-op, PvP",0,"windows, mac, linux","Free to Play: 1186, FPS: 757, Multiplayer: 755, Western: 702, Shooter: 640, Action: 535, First-Person: 507, Funny: 313, Indie: 281, Mod: 262, Online Co-Op: 246, Co-op: 212, Comedy: 182, Survival: 170, Massively Multiplayer: 153, Classic: 148, Historical: 142, Simulation: 117, Adventure: 110, RPG: 71",30986,2934,33920,0.913502
843380,Super Animal Royale,"Action, Adventure, Casual, Free to Play, Indie, Massively Multiplayer",game,"English, Japanese, Korean, Russian, Simplified Chinese, Traditional Chinese, French, Italian, German, Spanish - Spain, Spanish - Latin America, Portuguese - Brazil","Multi-player, Co-op, Full controller support, Online PvP, Online Co-op, PvP",0,"windows, mac","Battle Royale: 232, Cute: 187, Free to Play: 175, Multiplayer: 167, Massively Multiplayer: 141, Casual: 131, Shooter: 118, Action: 117, Indie: 114, 2D: 113, Top-Down Shooter: 102, Funny: 74, Adventure: 69, Early Access: 60, Co-op: 53, Online Co-Op: 53, Psychological Horror: 51, Survival: 51, Top-Down: 45, Family Friendly: 33",31189,1938,33127,0.941498
438040,Shakes and Fidget,"Adventure, Casual, Free to Play, Indie, Massively Multiplayer, RPG, Strategy",game,"English, French, Italian, German, Spanish - Spain, Bulgarian, Czech, Danish, Dutch, Finnish, Greek, Hungarian, Japanese, Norwegian, Polish, Portuguese - Portugal, Portuguese - Brazil, Romanian, Russian, Swedish, Turkish, Korean","Multi-player, Single-player, MMO, Cross-Platform Multiplayer, In-App Purchases",0,"windows, mac","Free to Play: 496, RPG: 269, Casual: 204, Massively Multiplayer: 202, Multiplayer: 185, Adventure: 174, Strategy: 166, Clicker: 135, MMORPG: 129, Fantasy: 122, Character Customization: 121, Indie: 105, Comedy: 83, Medieval: 81, Singleplayer: 65, Funny: 61, Classic: 59, Dark Humor: 54, Story Rich: 47, Cult Classic: 43",25639,1848,27487,0.932768
1397920,Mr.Mine,"Cas

### Platforms 별 추천

In [26]:
item.Platforms.unique()

array(['windows', 'windows, mac, linux', 'windows, mac', 'windows, linux'],
      dtype=object)

In [27]:
item[item.Platforms.str.contains('mac', na=False)].sort_values(by=['Reviews', 'PR_rate'], ascending=False).head(10).style.hide_index()
#['App ID'].values

App ID,Name,Genre,Type,Languages,Categories,Initial Price,Platforms,Tags,Positive Reviews,Negative Reviews,Reviews,PR_rate
4000,Garry's Mod,"Indie, Simulation",game,"English, French, Italian, German, Spanish - Spain, Bulgarian, Czech, Danish, Dutch, Finnish, Greek, Hungarian, Japanese, Korean, Norwegian, Polish, Portuguese - Portugal, Portuguese - Brazil, Russian, Simplified Chinese, Swedish, Thai, Traditional Chinese, Turkish, Ukrainian","Multi-player, Single-player, Valve Anti-Cheat enabled, Co-op, Captions available, Includes level editor, Steam Achievements, Steam Cloud, Cross-Platform Multiplayer, Steam Trading Cards, Steam Workshop, Online PvP, Online Co-op, Remote Play on Tablet, LAN PvP, LAN Co-op, PvP",999,"windows, mac, linux","Sandbox: 17628, Multiplayer: 13726, Funny: 12249, Moddable: 9861, Building: 9610, Comedy: 8298, Mod: 8239, Co-op: 8167, First-Person: 7937, Physics: 6867, Simulation: 6650, Online Co-Op: 6469, FPS: 6127, Singleplayer: 6055, Action: 5509, Animation & Modeling: 5339, Shooter: 5339, Indie: 4975, Adventure: 4651, Massively Multiplayer: 4650",861240,29998,891238,0.966341
227300,Euro Truck Simulator 2,"Indie, Simulation",game,"English, German, French, Italian, Korean, Spanish - Spain, Simplified Chinese, Russian, Japanese, Portuguese - Portugal, Polish, Danish, Dutch, Finnish, Norwegian, Swedish, Hungarian, Czech, Romanian, Turkish, Portuguese - Brazil, Bulgarian, Greek, Ukrainian","Multi-player, Single-player, Co-op, Includes level editor, Partial Controller Support, Steam Achievements, Steam Cloud, Steam Trading Cards, Steam Workshop, Online Co-op, Remote Play on Phone, Remote Play on Tablet, Remote Play Together",1999,"windows, mac, linux","Driving: 12059, Transportation: 12044, Simulation: 11989, Open World: 11951, Automobile Sim: 11939, Realistic: 11896, Moddable: 11836, Relaxing: 11834, Immersive Sim: 11783, Exploration: 11765, Economy: 11760, Management: 11743, Atmospheric: 11724, Multiplayer: 3978, Controller: 3740, Family Friendly: 3722, Third Person: 3721, Casual: 3655, Adventure: 3651, Indie: 3642",572368,15615,587983,0.973443
413150,Stardew Valley,"Indie, RPG, Simulation",game,"English, German, Spanish - Spain, Japanese, Portuguese - Brazil, Russian, Simplified Chinese, French, Italian, Hungarian, Korean, Turkish","Multi-player, Single-player, Co-op, Steam Achievements, Steam Cloud, Shared/Split Screen, Full controller support, Steam Trading Cards, Online Co-op, Shared/Split Screen Co-op, Remote Play on Phone, Remote Play on Tablet, Remote Play Together, LAN Co-op",1499,"windows, mac, linux","Farming Sim: 5804, Life Sim: 5607, Pixel Graphics: 5470, RPG: 5274, Multiplayer: 5058, Relaxing: 4623, Agriculture: 4445, Simulation: 4407, Crafting: 3638, Sandbox: 3398, Indie: 3115, Building: 2894, Singleplayer: 2635, Casual: 2546, Open World: 2467, 2D: 2249, Cute: 2082, Great Soundtrack: 2080, Dating Sim: 1910, Fishing: 1742",497558,9283,506841,0.981685
367520,Hollow Knight,"Action, Adventure, Indie",game,"English, French, German, Spanish - Spain, Korean, Simplified Chinese, Italian, Portuguese - Brazil, Russian, Japanese","Single-player, Steam Achievements, Steam Cloud, Full controller support, Steam Trading Cards, Remote Play on Phone, Remote Play on Tablet, Remote Play on TV",1499,"windows, mac, linux","Metroidvania: 4203, Souls-like: 3521, Platformer: 3214, Great Soundtrack: 3030, 2D: 2862, Difficult: 2860, Indie: 2498, Atmospheric: 2340, Singleplayer: 2285, Exploration: 2277, Adventure: 2117, Story Rich: 2028, Hand-drawn: 1571, Action: 1501, Dark Fantasy: 1394, Multiple Endings: 1382, Open World: 1262, Cute: 1224, Controller: 948, Side Scroller: 878",246135,7193,253328,0.971606
264710,Subnautica,"Adventure, Indie",game,"English, French, German, Polish, Russian, Simplified Chinese, Turkish, Finnish, Italian, Czech, Hungarian, Spanish - Spain, Danish, Japanese, Korean, Portuguese - Brazil, Bulgarian, Ukrainian, Dutch, Swedish, Vietnamese, Spanish - Latin America, Traditional Chinese, Portugues